# NLP Lab 6: Giới thiệu về Transformers

**Mục tiêu:**
- Ôn lại kiến thức cơ bản về kiến trúc Transformer
- Sử dụng các mô hình Transformer tiền huấn luyện (pretrained models) để thực hiện các tác vụ NLP cơ bản
- Làm quen với thư viện `transformers` của Hugging Face

**Nội dung:**
1. Bài 1: Khôi phục Masked Token (Masked Language Modeling)
2. Bài 2: Dự đoán từ tiếp theo (Next Token Prediction)
3. Bài 3: Tính toán Vector biểu diễn của câu (Sentence Representation)

## Bài 1: Khôi phục Masked Token (Masked Language Modeling)

Trong tác vụ này, chúng ta sẽ che một vài từ trong câu bằng token đặc biệt `[MASK]` và yêu cầu mô hình dự đoán từ gốc. Sử dụng mô hình thuộc họ BERT.

**Yêu cầu:** Sử dụng pipeline `fill-mask` để dự đoán từ bị thiếu trong câu: `Hanoi is the [MASK] of Vietnam.`

In [1]:
from transformers import pipeline

# 1. Tải pipeline "fill-mask"
# Pipeline này sẽ tự động tải một mô hình mặc định phù hợp (thường là một biến thể của BERT)
mask_filler = pipeline("fill-mask")

# 2. Câu đầu vào với token [MASK]
# Lưu ý: Một số model dùng <mask> thay vì [MASK]
input_sentence = "Hanoi is the <mask> of Vietnam."

# 3. Thực hiện dự đoán
# top_k=5 yêu cầu mô hình trả về 5 dự đoán hàng đầu
predictions = mask_filler(input_sentence, top_k=5)

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use mps:0


In [2]:
# 4. In kết quả
print(f"Câu gốc: {input_sentence}")
print("\nKết quả dự đoán:")
for pred in predictions:
    print(f"  Dự đoán: '{pred['token_str']}' với độ tin cậy: {pred['score']:.4f}")
    print(f"  -> Câu hoàn chỉnh: {pred['sequence']}")

Câu gốc: Hanoi is the <mask> of Vietnam.

Kết quả dự đoán:
  Dự đoán: ' capital' với độ tin cậy: 0.9341
  -> Câu hoàn chỉnh: Hanoi is the capital of Vietnam.
  Dự đoán: ' Republic' với độ tin cậy: 0.0300
  -> Câu hoàn chỉnh: Hanoi is the Republic of Vietnam.
  Dự đoán: ' Capital' với độ tin cậy: 0.0105
  -> Câu hoàn chỉnh: Hanoi is the Capital of Vietnam.
  Dự đoán: ' birthplace' với độ tin cậy: 0.0054
  -> Câu hoàn chỉnh: Hanoi is the birthplace of Vietnam.
  Dự đoán: ' heart' với độ tin cậy: 0.0014
  -> Câu hoàn chỉnh: Hanoi is the heart of Vietnam.


### Phân tích Bài 1:

**1. Mô hình đã dự đoán đúng từ `capital` không?**
- Có, mô hình dự đoán đúng từ "capital" với độ tin cậy rất cao (~0.93)
- Các dự đoán khác như "Republic", "Capital", "birthplace", "heart" cũng hợp lý về mặt ngữ pháp

**2. Tại sao các mô hình Encoder-only như BERT lại phù hợp cho tác vụ này?**
- BERT là mô hình **bidirectional** (hai chiều), có thể nhìn cả các từ đứng trước và sau token `[MASK]`
- Điều này cho phép mô hình hiểu ngữ cảnh đầy đủ của câu để dự đoán từ bị che
- Ví dụ: "Hanoi is the ___ of Vietnam" - mô hình cần hiểu cả "Hanoi" (thủ đô) và "Vietnam" (quốc gia) để dự đoán "capital"

## Bài 2: Dự đoán từ tiếp theo (Next Token Prediction)

Tác vụ này yêu cầu mô hình sinh ra phần tiếp theo của một đoạn văn bản cho trước. Sử dụng mô hình thuộc họ GPT.

**Yêu cầu:** Sử dụng pipeline `text-generation` để sinh ra phần tiếp theo cho câu: `The best thing about learning NLP is`

In [3]:
from transformers import pipeline

# 1. Tải pipeline "text-generation"
# Pipeline này sẽ tự động tải một mô hình phù hợp (thường là GPT-2)
generator = pipeline("text-generation")

# 2. Đoạn văn bản mồi
prompt = "The best thing about learning NLP is"

# 3. Sinh văn bản
# max_length: tổng độ dài của câu mồi và phần được sinh ra
# num_return_sequences: số lượng chuỗi kết quả muốn nhận
generated_texts = generator(prompt, max_length=50, num_return_sequences=1)

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use mps:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [4]:
# 4. In kết quả
print(f"Câu mồi: '{prompt}'")
print("\nVăn bản được sinh ra:")
for text in generated_texts:
    print(text['generated_text'])

Câu mồi: 'The best thing about learning NLP is'

Văn bản được sinh ra:
The best thing about learning NLP is that you learn a lot.

And I hope it helps you in other ways too.

Let's go back to the question: who can make your own version of Google?

I'm not sure if this is the right question to ask. I think that you will get a little bit of a feeling.

But let's look at a couple of examples:

What makes a good program? A good learning program is one that you can learn. This is not a prerequisite for anything, but it is a good way to keep pace. It can help you find the right tools and techniques. So let's say you have a computer program that you know you can learn by reading. Now, you might already have a good idea of what you are about to learn.

What could you do to catch up? And what would it be like to learn Google again?

I think that there are two reasons why this is a good question to ask. One is that people who know that they can learn Google will use it wisely. And the other is t

### Phân tích Bài 2:

**1. Kết quả sinh ra có hợp lý không?**
- Có, kết quả sinh ra có ngữ pháp đúng và mạch lạc
- Văn bản phát triển ý tưởng từ câu mồi một cách tự nhiên
- Lưu ý: Kết quả có thể khác nhau mỗi lần chạy do tính ngẫu nhiên của quá trình sinh

**2. Tại sao các mô hình Decoder-only như GPT lại phù hợp cho tác vụ này?**
- GPT là mô hình **unidirectional** (một chiều), chỉ nhìn các từ đã xuất hiện trước đó
- Cơ chế **auto-regressive**: dự đoán từ tiếp theo dựa vào các từ trước, rồi thêm từ đó vào input để dự đoán từ tiếp theo nữa
- Kiến trúc này hoàn toàn phù hợp với bản chất của việc sinh văn bản: tạo nội dung tuần tự từ trái sang phải

## Bài 3: Tính toán Vector biểu diễn của câu (Sentence Representation)

Một trong những ứng dụng mạnh mẽ nhất của BERT là khả năng chuyển đổi một câu thành một vector số có chiều dài cố định, nắm bắt được ngữ nghĩa của câu đó.

**Hai cách phổ biến để lấy vector biểu diễn:**
1. Lấy vector đầu ra của token `[CLS]`
2. Lấy trung bình cộng của các vector đầu ra (Mean Pooling) - thường cho kết quả tốt hơn

**Yêu cầu:** Viết code để tính toán vector biểu diễn cho câu `This is a sample sentence.` bằng phương pháp Mean Pooling.

In [5]:
import torch
from transformers import AutoTokenizer, AutoModel

# 1. Chọn một mô hình BERT
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. Câu đầu vào
sentences = ["This is a sample sentence."]

# 3. Tokenize câu
# padding=True: đệm các câu ngắn hơn để có cùng độ dài
# truncation=True: cắt các câu dài hơn
# return_tensors='pt': trả về kết quả dưới dạng PyTorch tensors
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

print("Tokens:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))
print("Input IDs:", inputs['input_ids'])
print("Attention Mask:", inputs['attention_mask'])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Tokens: ['[CLS]', 'this', 'is', 'a', 'sample', 'sentence', '.', '[SEP]']
Input IDs: tensor([[ 101, 2023, 2003, 1037, 7099, 6251, 1012,  102]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1]])


In [6]:
# 4. Đưa qua mô hình để lấy hidden states
# torch.no_grad() để không tính toán gradient, tiết kiệm bộ nhớ
with torch.no_grad():
    outputs = model(**inputs)

# outputs.last_hidden_state chứa vector đầu ra của tất cả các token
last_hidden_state = outputs.last_hidden_state
# shape: (batch_size, sequence_length, hidden_size)
print(f"Shape của last_hidden_state: {last_hidden_state.shape}")

Shape của last_hidden_state: torch.Size([1, 8, 768])


In [7]:
# 5. Thực hiện Mean Pooling
# Để tính trung bình chính xác, chúng ta cần bỏ qua các token đệm (padding tokens)
attention_mask = inputs['attention_mask']
mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
sentence_embedding = sum_embeddings / sum_mask

# 6. In kết quả
print("Vector biểu diễn của câu (10 phần tử đầu):")
print(sentence_embedding[0][:10])
print(f"\nKích thước của vector: {sentence_embedding.shape}")

Vector biểu diễn của câu (10 phần tử đầu):
tensor([-6.3875e-02, -4.2837e-01, -6.6779e-02, -3.8430e-01, -6.5785e-02,
        -2.1826e-01,  4.7636e-01,  4.8659e-01,  3.9354e-05, -7.4273e-02])

Kích thước của vector: torch.Size([1, 768])


### Phân tích Bài 3:

**1. Kích thước (chiều) của vector biểu diễn là bao nhiêu? Con số này tương ứng với tham số nào của mô hình BERT?**
- Kích thước vector là **768 chiều**
- Con số này tương ứng với tham số `hidden_size` của mô hình `bert-base-uncased`
- Các biến thể khác: `bert-large` có hidden_size = 1024

**2. Tại sao chúng ta cần sử dụng `attention_mask` khi thực hiện Mean Pooling?**
- Khi xử lý batch, các câu ngắn hơn được thêm **padding tokens** để có cùng độ dài
- Padding tokens không mang ý nghĩa ngữ nghĩa
- Nếu đưa vào phép tính trung bình, kết quả sẽ bị sai lệch
- `attention_mask` giúp "che" các padding tokens, chỉ tính trung bình trên các token thực sự

## Ứng dụng: So sánh độ tương đồng giữa các câu

Sử dụng sentence embeddings để tính cosine similarity giữa các câu.

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_sentence_embedding(sentence, tokenizer, model):
    """Tính sentence embedding bằng Mean Pooling"""
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    
    last_hidden_state = outputs.last_hidden_state
    attention_mask = inputs['attention_mask']
    mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
    sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
    sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
    return (sum_embeddings / sum_mask).numpy()

# Các câu để so sánh
sentences = [
    "I love machine learning.",
    "I enjoy studying artificial intelligence.",
    "The weather is nice today.",
    "It is sunny outside."
]

# Tính embeddings
embeddings = np.vstack([get_sentence_embedding(s, tokenizer, model) for s in sentences])

# Tính cosine similarity
similarity_matrix = cosine_similarity(embeddings)

print("Ma trận độ tương đồng:")
print("\nCâu:")
for i, s in enumerate(sentences):
    print(f"  [{i}] {s}")

print("\nSimilarity Matrix:")
print(np.round(similarity_matrix, 3))

Ma trận độ tương đồng:

Câu:
  [0] I love machine learning.
  [1] I enjoy studying artificial intelligence.
  [2] The weather is nice today.
  [3] It is sunny outside.

Similarity Matrix:
[[1.    0.863 0.589 0.619]
 [0.863 1.    0.607 0.634]
 [0.589 0.607 1.    0.839]
 [0.619 0.634 0.839 1.   ]]


### Phân tích kết quả:
- Câu [0] và [1] có độ tương đồng cao vì cùng chủ đề về AI/ML
- Câu [2] và [3] có độ tương đồng cao vì cùng chủ đề về thời tiết
- Các cặp câu khác chủ đề có độ tương đồng thấp hơn